<a href="https://colab.research.google.com/github/Da-Debanes/Portfolio-1/blob/main/FSC_Group_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**OBJECTIVE**

###**Input:**
A data set with participants' information in it.

###**Output:**

People assigned into groupings based on personality, availability and gender.

 This information will be in a sheet where the first tab is a summary of the peoples name in each group of all groups and the subsequent tabs are the full information of everyone who is in a particular group with one tab per group

## 1. Imported Libraries
We will refer to the Pandas, SQL and CSV libraries in order to facilitate the code.

In [338]:
import pandas as pd
import random
from collections import deque
#!pip install XlsxWriter
import math


## 2. File **Input**
Remember to specify the input file name. Should be a csv.

In [340]:
fayil = "FSC Signups.csv"
df = pd.read_csv(fayil)

## 4. Grouping
Participants will thus be assigned into groups based on what their personality and gender is

In [341]:
df["Personality"] = df["Personality"].fillna("M XXX")
df["Pre-U Education"] =  df["Pre-U Education"].fillna("Others")

personalityname = "Personality"
gendername = "Gender"

df["Personality_Gender"] = df[personalityname] + "-" + df[gendername] + "-" + df["Course of study :"]

# Group by personality
personality_groups = {
    ptype: df[df["Personality_Gender"] == ptype].copy()
    for ptype in df["Personality_Gender"].unique()
}


In [342]:
df["Personality_Gender"].value_counts().sum()

np.int64(228)

##3. Assigning Genders


In [343]:
male_df = df[df['Gender'] == "Male"]
female_df = df[df['Gender'] == "Female"]

personality_groups_male = {
    ptype: male_df[male_df["Personality_Gender"] == ptype].copy()
    for ptype in male_df["Personality_Gender"].unique()
}

personality_groups_female = {
    ptype: female_df[female_df["Personality_Gender"] == ptype].copy()
    for ptype in female_df["Personality_Gender"].unique()
}

In [344]:
male_df['Personality_Gender'].value_counts().sum()

np.int64(179)

##5. Aggregate Probability Distribution

In [345]:
distribution = df["Personality_Gender"].value_counts(normalize=True)

##6. Grouping
A for loop goes through all the data sets and put the id of a person to their corresponding personality-gender set. Then given the probability distribution, the sorting will select a set to get a single person from and remove that person from their set

In [346]:
id_queues_by_group_female.clear()
id_queues_by_group_male.clear()

id_queues_by_group_male = {
    ptype: deque(random.sample(df["Id"].tolist(), len(df)))
    for ptype, df in personality_groups_male.items()
}

id_queues_by_group_female = {
    ptype: deque(random.sample(group_df["Id"].tolist(), len(group_df)))
    for ptype, group_df in personality_groups_female.items()
}

In [347]:
sum(len(queue) for queue in id_queues_by_group_male.values())

179

In [348]:
def draw_person(id_queues_by_group):
    group_names = list(id_queues_by_group.keys())
    weights = [len(q) for q in id_queues_by_group.values()]

    if not group_names or sum(weights) == 0:
        return None, None

    selected_group = random.choices(group_names, weights=weights, k=1)[0]
    person_id = id_queues_by_group[selected_group].popleft()

    if not id_queues_by_group[selected_group]:
        del id_queues_by_group[selected_group]

    return selected_group, person_id


In [349]:
group_size = 10  # Replace with your desired group size
total_students = len(df)
n_groups = total_students // group_size + (1 if total_students % group_size > 0 else 0)

groups = []
id_to_group = {}

male_ratio = sum(len(queue) for queue in id_queues_by_group_male.values()) / total_students
female_ratio = sum(len(queue) for queue in id_queues_by_group_female.values()) / total_students

males_per_group = math.floor(male_ratio * group_size + 0.5)
females_per_group = group_size - males_per_group

for i in range(n_groups - 1):
    group_ids = []

    for j in range(females_per_group):
        k, person_id = draw_person(id_queues_by_group_female)
        if person_id:
            group_ids.append(person_id)
            id_to_group[person_id] = i + 1

    for j in range(males_per_group):
        k, person_id = draw_person(id_queues_by_group_male)
        if person_id:
            group_ids.append(person_id)
            id_to_group[person_id] = i + 1

    groups.append(group_ids)

remaining_ids = []
for group_dict in [id_queues_by_group_male, id_queues_by_group_female]:
    for queue in group_dict.values():
        remaining_ids.extend(queue)
for i in remaining_ids:
    id_to_group[i] = n_groups
groups.append(remaining_ids)

In [350]:
gsum = sum(len(group) for group in groups)
print(gsum)
print(groups[-1])
print(id_queues_by_group_female)
print(id_queues_by_group_male)

228
[12, 144, 168, 183, 174, 159, 157, 125]
{'I XXX-Female-Business Analytics': deque([183]), 'E XXX-Female-Business Analytics': deque([174]), 'I XXX-Female-Computer Science': deque([159, 157]), 'I XXX-Female-Information Security': deque([125])}
{'M XXX-Male-Computer Science': deque([12]), 'E XXX-Male-Business Analytics': deque([144]), 'I XXX-Male-Information Systems': deque([168])}


##7. Writing the group allocations

In [351]:
id_to_group = {}

for i in range(n_groups):
    for id in groups[i]:
        id_to_group[id] = i + 1

df["Ori Grouping"] = df["Id"].map(id_to_group)

In [352]:
print("Unassigned:", df["Ori Grouping"].isna().sum())


Unassigned: 0


##8. Writing into Excel

In [357]:
#TODO Replace these with your actual column names
summary_cols = ["Ori Grouping", "Name", "Student/Application Number", "Telegram handle (eg @shr221)"]
shirt_size_col = "Shirt Size"
Maj_col = "Course of study :"

summary_df = df[summary_cols].sort_values(by="Ori Grouping")

shirt_summary = df.groupby(["Ori Grouping", shirt_size_col]).size().unstack(fill_value=0)
gender_summary = df.groupby(["Ori Grouping", "Gender"]).size().unstack(fill_value=0)
Maj_summary = df.groupby(["Ori Grouping", Maj_col]).size().unstack(fill_value=0)

with pd.ExcelWriter("grouped_participants.xlsx", engine="xlsxwriter") as writer:

    summary_df.to_excel(writer, sheet_name="Summary", index=False)
    shirt_summary.to_excel(writer, sheet_name="Shirt Size Summary")
    gender_summary.to_excel(writer, sheet_name="Gender Summary")
    Maj_summary.to_excel(writer, sheet_name="Maj Summary")

    for group_name, group_df in df.groupby("Ori Grouping"):
        sheet_name = str(group_name)
        group_df.to_excel(writer, sheet_name=sheet_name, index=False)

